In [1]:
!pip install parameters

You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle
import nest
nest.set_verbosity('M_ERROR')


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 19 2024 15:53:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
net_dict_filename = 'network_parameters.pkl'
with open(net_dict_filename, 'rb') as f:
    net_dict = pickle.load(f)

In [ ]:
#L2/3E, L2/3I, L4E, L4I, L5E, L5I, L6E, L6I, TC

In [4]:
net_dict['num_neurons']

array([ 758176,  213856, 1126192,  281552,  331840,   72864,  317424,
         65008,   14432])

In [ ]:
#L2E: 80
#L2I: 10
#L3E: 80
#L3I: 10
#L4E: 120
#L4I: 28
#L5E: 60
#L5I: 20
#L6E: 90
#L6I: 20

In [7]:
N_scaling = 0.001 #for 1mm^3

N_L2E = np.round(net_dict['num_neurons'][0] * N_scaling).astype(int)
N_L2I = np.round(net_dict['num_neurons'][1] * N_scaling).astype(int)
N_L3E = np.round(net_dict['num_neurons'][0] * N_scaling).astype(int)
N_L3I = np.round(net_dict['num_neurons'][1] * N_scaling).astype(int)
N_L4E = np.round(net_dict['num_neurons'][2] * N_scaling).astype(int)
N_L4I = np.round(net_dict['num_neurons'][3] * N_scaling).astype(int)
N_L5E = np.round(net_dict['num_neurons'][4] * N_scaling).astype(int)
N_L5I = np.round(net_dict['num_neurons'][5] * N_scaling).astype(int)
N_L6E = np.round(net_dict['num_neurons'][6] * N_scaling).astype(int)
N_L6I = np.round(net_dict['num_neurons'][7] * N_scaling).astype(int)

N_L2E, N_L2I, N_L3E, N_L3I, N_L4E, N_L4I, N_L5E, N_L5I, N_L6E, N_L6I

(758, 214, 758, 214, 1126, 282, 332, 73, 317, 65)

In [8]:
extent = net_dict['extent']

extent

4.0

In [10]:
neuron_params = {
    # membrane capacitance (in pF)
    'C_m': 250.0,
    # membrane time constant (in ms)
    'tau_m': 10.0,    
    # resistive leak reversal potential (in mV)
    'E_L': -65.0,
    # spike detection threshold (in mV)
    'V_th': -50.0,
    # spike reset potential (in mV)
    'V_reset': -65.0,
    # refractory period after spikes (in ms)
    't_ref': 2.0,
    # time constants of postsynaptic currents (in ms)
    'tau_syn_ex': 2.0,
    'tau_syn_in': 8.0}

In [9]:
net_left_cortex = net_dict
net_right_cortex = net_dict

In [13]:
#net_left_cortex['indegrees']
#beta_L6I_L6E = net_left_cortex['beta'][6, 7] # L6I -> L6E
#net_left_cortex['weight_matrix_mean']
#delay_offset = net_left_cortex['delay_offset_exc_inh'][0]
#prop_speed = net_left_cortex['prop_speed_exc_inh'][0]
#bg_rate = net_left_cortex['bg_rate']
delay_poisson = 1.0

In [15]:
use_thalamic_pulse = True #the thalamus only sends to layer 4,5,6
th_radius = 0.5
th_delay = 1.0
th_weight = 500 * 1 #gotta change this weight
th_rate = 100.
th_start = 500.
th_stop = 550.

In [16]:
nest.ResetKernel()
nest.rng_seed = 42
nest.resolution = 0.1
nest.local_num_threads = os.cpu_count()

In [17]:
positions = nest.spatial.free(
    pos=nest.random.uniform(min=-extent / 2.,
                            max=extent / 2.),
    edge_wrap=True,
    extent=[extent, extent])